In [34]:
import pulp
from dataclasses import dataclass, field
from typing import List, Dict, Optional
import json
from datetime import datetime, timedelta, date
import pandas as pd

In [ ]:
# 1. DEFINICIÓN DE ESTRUCTURAS DE DATOS (Adaptado de data_models.py)
# --------------------------------------------------------------------
@dataclass
class Resource:
    name: str
    availability: Dict[str, float] = field(default_factory=dict)

@dataclass
class Task:
    id: str
    name: str
    project_id: str
    hours: float
    sequence: int
    compatible_resources: List[str] = field(default_factory=list)
    subcontractable: bool = False
    # --- CAMPOS NUEVOS AÑADIDOS ---
    requires_client_validation: bool = False
    validation_delay_days: int = 0
    #------------------------
    predecessors: List[str] = field(default_factory=list)

@dataclass
class Project:
    id: str
    name: str


In [37]:
# 3. CLASE DEL OPTIMIZADOR CON PULP
# --------------------------------------------------------------------
class OptimizadorProyectos:
    """
    Implementa el modelo de optimización para maximizar proyectos completados.
    """
    def __init__(self, proyectos, tareas, recursos, params):
        self.proyectos = proyectos
        self.tareas = tareas
        self.recursos = recursos
        self.params = params
        
        # Estructuras de datos para fácil acceso
        self.task_map = {t.id: t for t in self.tareas}
        
        # Modelo PuLP
        self.model = pulp.LpProblem("Maximizacion_Proyectos", pulp.LpMaximize)
        self.vars = {}

    def construir_modelo(self):
        """Traduce el modelo matemático a código PuLP."""
        H = self.params["horizonte_dias"]
        DIAS = list(range(1, H + 1))
        
        # --- Variables de Decisión ---
        self.vars['x'] = pulp.LpVariable.dicts("horas",
            ((t.id, r.name, d) for t in self.tareas for r in self.recursos for d in DIAS),
            lowBound=0, cat='Continuous')
        
        self.vars['s'] = pulp.LpVariable.dicts("inicio_tarea", (t.id for t in self.tareas), lowBound=1, cat='Integer')
        self.vars['e'] = pulp.LpVariable.dicts("fin_tarea", (t.id for t in self.tareas), lowBound=1, cat='Integer')
        
        self.vars['SubT'] = pulp.LpVariable.dicts("subcontrata_tarea", (t.id for t in self.tareas), cat='Binary')
        self.vars['C'] = pulp.LpVariable.dicts("completa_proyecto", (p.id for p in self.proyectos), cat='Binary')
        self.vars['SubP'] = pulp.LpVariable.dicts("subcontrata_proyecto", (p.id for p in self.proyectos), cat='Binary')
        
        # Variables para linealización del objetivo
        self.vars['CI'] = pulp.LpVariable.dicts("completa_interno", (p.id for p in self.proyectos), cat='Binary')
        self.vars['CS'] = pulp.LpVariable.dicts("completa_subcontratado", (p.id for p in self.proyectos), cat='Binary')
        
        # --- Función Objetivo ---
        alpha = self.params['alpha']
        beta = self.params['beta']
        self.model += pulp.lpSum(alpha * self.vars['CI'][p.id] + beta * self.vars['CS'][p.id] for p in self.proyectos), "Beneficio_Total_Proyectos"

        # --- Restricciones ---
        
        # 1. Dedicación de Horas
        for t in self.tareas:
            self.model += pulp.lpSum(self.vars['x'][t.id, r.name, d] for r in self.recursos if r.name in t.compatible_resources for d in DIAS) == t.hours * (1 - self.vars['SubT'][t.id]), f"Horas_Tarea_{t.id}"

        # 2. Disponibilidad de Recursos
        for r in self.recursos:
            for d in DIAS:
                # Mapear día numérico a nombre de día
                dia_semana = self.params["dias_laborables"][(d-1) % len(self.params["dias_laborables"])]
                disponibilidad = r.availability.get(dia_semana, 0)
                self.model += pulp.lpSum(self.vars['x'][t.id, r.name, d] for t in self.tareas) <= disponibilidad, f"Disponibilidad_{r.name}_{d}"
        
        # 3. Secuencialidad
        for t in self.tareas:
            for pred_id in t.predecessors:
                self.model += self.vars['s'][t.id] >= self.vars['e'][pred_id] + 1, f"Secuencia_{pred_id}_{t.id}"

        # 4. Lógica de días de inicio y fin (versión simplificada)
        M = H * 10 # Big M
        for t in self.tareas:
            for d in DIAS:
                # Si se trabaja en el día d, el día de fin es al menos d
                self.model += self.vars['e'][t.id] >= d * pulp.lpSum(self.vars['x'][t.id, r.name, d] for r in self.recursos) / M, f"Def_Fin_{t.id}_{d}"
            # El inicio es antes o igual que el fin
            self.model += self.vars['s'][t.id] <= self.vars['e'][t.id], f"Inicio_antes_de_Fin_{t.id}"

        # 5. Completitud del Proyecto
        for p in self.proyectos:
            tareas_del_proyecto = [t for t in self.tareas if t.project_id == p.id]
            if tareas_del_proyecto:
                # El proyecto se completa si todas sus tareas finalizan dentro del horizonte
                for t in tareas_del_proyecto:
                     self.model += t.hours <= M * self.vars['C'][p.id] + M * self.vars['SubT'][t.id], f"Completitud_{p.id}_{t.id}"
                     #self.model += self.vars['e']

    def resolver(self):
            """Resuelve el modelo y devuelve el estado."""
            self.model.solve()
            return pulp.LpStatus[self.model.status]

    def mostrar_resultados(self):
        """Imprime los resultados de la optimización en la consola."""
        print("\n--- RESULTADOS DE LA OPTIMIZACIÓN ---")
        print(f"Estado: {pulp.LpStatus[self.model.status]}")
        print(f"Beneficio Total Óptimo: {pulp.value(self.model.objective):.2f}")
            
        print("\n**Estado de los Proyectos:**")
        for p in self.proyectos:
            if pulp.value(self.vars['C'][p.id]) == 1:
                tipo = "Subcontratado" if pulp.value(self.vars['SubP'][p.id]) == 1 else "Interno"
                print(f"- Proyecto '{p.name}': COMPLETADO ({tipo})")
            else:
                print(f"- Proyecto '{p.name}': NO COMPLETADO")

        print("\n**Detalle de Tareas:**")
        for t in self.tareas:
            if pulp.value(self.vars['SubT'][t.id]) == 1:
                print(f"- Tarea '{t.name}' ({t.project_id}): SUBCONTRATADA")
            else:
                inicio = pulp.value(self.vars['s'][t.id])
                fin = pulp.value(self.vars['e'][t.id])
                print(f"- Tarea '{t.name}' ({t.project_id}): Inicia día {inicio:.0f}, Finaliza día {fin:.0f}")

        print("\n**Asignación de Horas (resumen):**")
        for t in self.tareas:
            if pulp.value(self.vars['SubT'][t.id]) == 0:
                for r in self.recursos:
                    horas_asignadas = sum(pulp.value(self.vars['x'][t.id, r.name, d]) for d in range(1, self.params['horizonte_dias'] + 1))
                    if horas_asignadas > 0:
                        print(f"  - Recurso '{r.name}' trabaja {horas_asignadas:.1f}h en la tarea '{t.name}'")


    @classmethod
    def desde_json(cls, ruta_fichero: str):
        """
        Método de clase para crear una instancia del optimizador a partir de un fichero JSON.
        """
        print(f"Cargando instancia desde: {ruta_fichero}")
        with open(ruta_fichero, 'r') as f:
            data = json.load(f)

        # Cargar datos en las dataclasses
        proyectos = [Project(**p) for p in data['proyectos']]
        tareas = [Task(**t) for t in data['tareas']]
        recursos = [Resource(**r) for r in data['recursos']]
        
        # Procesar parámetros y convertir fecha
        params = data['parametros']
        params['fecha_inicio'] = datetime.strptime(params['fecha_inicio'], '%Y-%m-%d').date()

        # Crear y devolver una instancia de la clase
        return cls(proyectos, tareas, recursos, params)
    
    def visualizar_gantt(self):
        if self.model.status != pulp.LpStatusOptimal:
            print("\nNo se puede generar el Gantt: no se encontró una solución óptima.")
            return

        fecha_inicio = self.params["fecha_inicio"]
        gantt_data = []

        for t in self.tareas:
            if pulp.value(self.vars['SubT'][t.id]) == 0:
                start_day = pulp.value(self.vars['s'][t.id])
                finish_day = pulp.value(self.vars['e'][t.id])
                
                assigned_resource = "N/A"
                for r in self.recursos:
                    if sum(pulp.value(self.vars['x'][t.id, r.name, d]) for d in range(1, self.params['horizonte_dias'] + 1)) > 0:
                        assigned_resource = r.name
                        break
                
                gantt_data.append(dict(
                    Task=f"{self.project_map[t.project_id].name}: {t.name}",
                    Start=(fecha_inicio + timedelta(days=start_day-1)),
                    Finish=(fecha_inicio + timedelta(days=finish_day)),
                    Project=self.project_map[t.project_id].name,
                    Resource=assigned_resource
                ))

        if not gantt_data:
            print("\nNo hay tareas internas planificadas para mostrar en el Gantt.")
            return
            
        df = pd.DataFrame(gantt_data)
        
        fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task", color="Project",
                          title="Diagrama de Gantt de la Planificación",
                          hover_data=["Resource"])
        
        fig.update_yaxes(categoryorder="total ascending")
        fig.show()

In [32]:
if __name__ == "__main__":
    # Nombre del fichero con la instancia a resolver
    fichero_instancia = r'..\instancias\instancia_mediana.json'
    
    # 1. Crear el optimizador cargando los datos desde el fichero JSON
    optimizador = OptimizadorProyectos.desde_json(fichero_instancia)
    
    # 2. Construir el modelo
    print("Construyendo el modelo de optimización...")
    optimizador.construir_modelo()
    
    # 3. Resolver
    print("Resolviendo... (esto puede tardar unos segundos o minutos)")
    estado = optimizador.resolver()
    
    # 4. Mostrar resultados
    if estado == 'Optimal':
        optimizador.mostrar_resultados()
        print("\nGenerando diagrama de Gantt...")
        optimizador.visualizar_gantt()
    else:
        print(f"\nNo se encontró una solución óptima. Estado: {estado}")

Cargando instancia desde: ..\instancias\instancia_mediana.json
Construyendo el modelo de optimización...
Resolviendo... (esto puede tardar unos segundos o minutos)

--- RESULTADOS DE LA OPTIMIZACIÓN ---
Estado: Optimal
Beneficio Total Óptimo: 600.00

**Estado de los Proyectos:**
- Proyecto 'SimulaciÃ³n Estructural': COMPLETADO (Interno)
- Proyecto 'AnÃ¡lisis CFD Ala': COMPLETADO (Interno)
- Proyecto 'Desarrollo Dispositivo IoT': COMPLETADO (Interno)
- Proyecto 'Mantenimiento Interno Anual': COMPLETADO (Interno)

**Detalle de Tareas:**
- Tarea 'Setup CAD' (PROJ-SIM-01): SUBCONTRATADA
- Tarea 'Mallado FEM' (PROJ-SIM-01): SUBCONTRATADA
- Tarea 'EjecuciÃ³n SimulaciÃ³n' (PROJ-SIM-01): SUBCONTRATADA
- Tarea 'Post-proceso e Informe' (PROJ-SIM-01): SUBCONTRATADA
- Tarea 'Limpieza GeometrÃ­a' (PROJ-CFD-02): SUBCONTRATADA
- Tarea 'GeneraciÃ³n Volumen Control' (PROJ-CFD-02): SUBCONTRATADA
- Tarea 'Mallado CFD' (PROJ-CFD-02): SUBCONTRATADA
- Tarea 'Informe CFD' (PROJ-CFD-02): SUBCONTRATADA
- Tarea